In [1]:
!python3.6 -m pip install tensorboardX torch librosa tqdm lws
!apt-get install libsndfile1 --yes

     |████████████████████████████████| 204kB 1.1MB/s eta 0:00:01
     |████████████████████████████████| 734.6MB 106kB/s  eta 0:00:01   |▊                               | 17.8MB 8.5MB/s eta 0:01:25     |████████▌                       | 195.4MB 12.3MB/s eta 0:00:44     |███████████▉                    | 270.7MB 12.0MB/s eta 0:00:39     |███████████████▌                | 354.5MB 10.8MB/s eta 0:00:36     |█████████████████████▎          | 489.0MB 12.2MB/s eta 0:00:21 eta 0:00:12��███████████████   | 665.6MB 12.1MB/s eta 0:00:06
     |████████████████████████████████| 1.6MB 5.3MB/s eta 0:00:01
     |████████████████████████████████| 61kB 8.4MB/s  eta 0:00:01
     |████████████████████████████████| 153kB 11.9MB/s eta 0:00:01
     |████████████████████████████████| 26.1MB 12.7MB/s eta 0:00:01
     |████████████████████████████████| 7.1MB 11.2MB/s eta 0:00:01
     |████████████████████████████████| 296kB 12.5MB/s eta 0:00:01
     |████████████████████████████████| 327kB 16.8MB/s eta 0:00:01

Get:5 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libsndfile1 amd64 1.0.28-4ubuntu0.18.04.1 [170 kB]
Fetched 557 kB in 1s (1046 kB/s)    
debconf: delaying package configuration, since apt-utils is not installed
Selecting previously unselected package libogg0:amd64.
(Reading database ... 15402 files and directories currently installed.)
Preparing to unpack .../libogg0_1.3.2-1_amd64.deb ...
Unpacking libogg0:amd64 (1.3.2-1) ...
Selecting previously unselected package libflac8:amd64.
Preparing to unpack .../libflac8_1.3.2-1_amd64.deb ...
Unpacking libflac8:amd64 (1.3.2-1) ...
Selecting previously unselected package libvorbis0a:amd64.
Preparing to unpack .../libvorbis0a_1.3.5-4.2_amd64.deb ...
Unpacking libvorbis0a:amd64 (1.3.5-4.2) ...
Selecting previously unselected package libvorbisenc2:amd64.
Preparing to unpack .../libvorbisenc2_1.3.5-4.2_amd64.deb ...
Unpacking libvorbisenc2:amd64 (1.3.5-4.2) ...
Selecting previously unselected package libsndfile1:amd64.
Preparing to 

# Test code copy

In [3]:
import torch
from torch import optim
from torch.autograd import Variable
import numpy as np
import pickle
from utils import Hps
from preprocess.tacotron.norm_utils import spectrogram2wav, get_spectrogram
from scipy.io.wavfile import write
import glob
import os
from solver import Solver
import librosa
import librosa.display
from plots import plot
%matplotlib inline

args_hps='vctk.json'
args_model="model_vctk/fixed_8bit_16k_paused_reusmed_latschediters_50k/model_vctk-80000"
args_source="test/p225_001.wav"
args_source_8="test/p225_001-8.wav"
args_target=0 # testuj na takim co miał dużo >128, zapisz output sobie najlepiej
args_output="test/output.wav"
args_sample_rate=16000
args_use_gen=False
wavenet_mel=False

hps = Hps()
hps.load(args_hps)
hps_tuple = hps.get_tuple()
solver = Solver(hps_tuple, None, wavenet_mel=wavenet_mel)
solver.load_model(args_model)
spec = get_spectrogram(args_source, wavenet_mel)
spec_8 = get_spectrogram(args_source_8, wavenet_mel)

spec_expand = np.expand_dims(spec_8, axis=0)
spec_tensor = torch.from_numpy(spec_expand).type(torch.FloatTensor)
c = Variable(torch.from_numpy(np.array([int(args_target)]))).cuda()
result = solver.test_step(spec_tensor, c, gen=args_use_gen)
result = result.squeeze(axis=0).transpose((1, 0))
wav_data = spectrogram2wav(result)
write(args_output, rate=args_sample_rate, data=wav_data)

plot(spec.T, spec_8.T, result.T)

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


# Train code copy

In [ ]:
import torch
from torch import optim
from torch.autograd import Variable
import numpy as np
import pickle
from utils import Hps
from utils import DataLoader
from utils import Logger
from utils import SingleDataset
from solver import Solver
import argparse
%matplotlib inline

args_load_model=False
args_flag='train'
args_hps_path="/tf/notebooks/SKAJPAI/voice_style_transfer/voice_conversion/vctk.json"
args_dataset_path_trg="/tf/notebooks/SKAJPAI/voice_style_transfer/voice_conversion/vctk_old/data-wavenet.h5"
args_dataset_path="/tf/notebooks/SKAJPAI/voice_style_transfer/voice_conversion/vctk_old/data-8-wavenet.h5"
args_index_path_trg="/tf/notebooks/SKAJPAI/voice_style_transfer/voice_conversion/vctk_old/index-wavenet.json"
args_index_path="/tf/notebooks/SKAJPAI/voice_style_transfer/voice_conversion/vctk_old/index-wavenet.json"
args_output_model_path="/tf/notebooks/SKAJPAI/voice_style_transfer/voice_conversion/model_vctk/model_vctk"
args_load_model_path="model_vctk/model_vctk-32000"
wavenet_mel=True

hps = Hps()
hps.load(args_hps_path)
hps_tuple = hps.get_tuple()
dataset = SingleDataset(args_dataset_path, args_index_path, seg_len=hps_tuple.seg_len)
dataset_trg = SingleDataset(args_dataset_path_trg, args_index_path_trg, seg_len=hps_tuple.seg_len)
data_loader = DataLoader(dataset, dataset_trg)

solver = Solver(hps_tuple, data_loader, wavenet_mel)
if args_load_model:
    solver.load_model(args_load_model_path)

solver.train(args_output_model_path, args_flag, mode='pretrain_G')
solver.train(args_output_model_path, args_flag, mode='pretrain_D')
solver.train(args_output_model_path, args_flag, mode='train')
solver.train(args_output_model_path, args_flag, mode='patchGAN')

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2404: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


pre_G:[000001/008000], loss_rec=0.440
pre_G:[000002/008000], loss_rec=0.422
pre_G:[000003/008000], loss_rec=0.370
pre_G:[000004/008000], loss_rec=0.340
pre_G:[000005/008000], loss_rec=0.332
pre_G:[000006/008000], loss_rec=0.293
pre_G:[000007/008000], loss_rec=0.278
pre_G:[000008/008000], loss_rec=0.276
pre_G:[000009/008000], loss_rec=0.254
pre_G:[000010/008000], loss_rec=0.260
pre_G:[000011/008000], loss_rec=0.242
pre_G:[000012/008000], loss_rec=0.232
pre_G:[000013/008000], loss_rec=0.232
pre_G:[000014/008000], loss_rec=0.219
pre_G:[000015/008000], loss_rec=0.219
pre_G:[000016/008000], loss_rec=0.216
pre_G:[000017/008000], loss_rec=0.202
pre_G:[000018/008000], loss_rec=0.200
pre_G:[000019/008000], loss_rec=0.199
pre_G:[000020/008000], loss_rec=0.194
pre_G:[000021/008000], loss_rec=0.197
pre_G:[000022/008000], loss_rec=0.189
pre_G:[000023/008000], loss_rec=0.190
pre_G:[000024/008000], loss_rec=0.180
pre_G:[000025/008000], loss_rec=0.188
pre_G:[000026/008000], loss_rec=0.184
pre_G:[00002

pre_G:[000220/008000], loss_rec=0.080
pre_G:[000221/008000], loss_rec=0.075
pre_G:[000222/008000], loss_rec=0.077
pre_G:[000223/008000], loss_rec=0.080
pre_G:[000224/008000], loss_rec=0.076
pre_G:[000225/008000], loss_rec=0.076
pre_G:[000226/008000], loss_rec=0.075
pre_G:[000227/008000], loss_rec=0.081
pre_G:[000228/008000], loss_rec=0.084
pre_G:[000229/008000], loss_rec=0.078
pre_G:[000230/008000], loss_rec=0.073
pre_G:[000231/008000], loss_rec=0.086
pre_G:[000232/008000], loss_rec=0.084
pre_G:[000233/008000], loss_rec=0.077
pre_G:[000234/008000], loss_rec=0.075
pre_G:[000235/008000], loss_rec=0.075
pre_G:[000236/008000], loss_rec=0.073
pre_G:[000237/008000], loss_rec=0.081
pre_G:[000238/008000], loss_rec=0.082
pre_G:[000239/008000], loss_rec=0.074
pre_G:[000240/008000], loss_rec=0.079
pre_G:[000241/008000], loss_rec=0.080
pre_G:[000242/008000], loss_rec=0.075
pre_G:[000243/008000], loss_rec=0.077
pre_G:[000244/008000], loss_rec=0.079
pre_G:[000245/008000], loss_rec=0.079
pre_G:[00024

pre_G:[000438/008000], loss_rec=0.064
pre_G:[000439/008000], loss_rec=0.068
pre_G:[000440/008000], loss_rec=0.069
pre_G:[000441/008000], loss_rec=0.070
pre_G:[000442/008000], loss_rec=0.070
pre_G:[000443/008000], loss_rec=0.072
pre_G:[000444/008000], loss_rec=0.070
pre_G:[000445/008000], loss_rec=0.072
pre_G:[000446/008000], loss_rec=0.072
pre_G:[000447/008000], loss_rec=0.070
pre_G:[000448/008000], loss_rec=0.066
pre_G:[000449/008000], loss_rec=0.066
pre_G:[000450/008000], loss_rec=0.077
pre_G:[000451/008000], loss_rec=0.066
pre_G:[000452/008000], loss_rec=0.070
pre_G:[000453/008000], loss_rec=0.067
pre_G:[000454/008000], loss_rec=0.071
pre_G:[000455/008000], loss_rec=0.068
pre_G:[000456/008000], loss_rec=0.065
pre_G:[000457/008000], loss_rec=0.065
pre_G:[000458/008000], loss_rec=0.065
pre_G:[000459/008000], loss_rec=0.066
pre_G:[000460/008000], loss_rec=0.067
pre_G:[000461/008000], loss_rec=0.064
pre_G:[000462/008000], loss_rec=0.069
pre_G:[000463/008000], loss_rec=0.068
pre_G:[00046

pre_G:[000655/008000], loss_rec=0.062
pre_G:[000656/008000], loss_rec=0.060
pre_G:[000657/008000], loss_rec=0.066
pre_G:[000658/008000], loss_rec=0.064
pre_G:[000659/008000], loss_rec=0.060
pre_G:[000660/008000], loss_rec=0.058
pre_G:[000661/008000], loss_rec=0.061
pre_G:[000662/008000], loss_rec=0.064
pre_G:[000663/008000], loss_rec=0.060
pre_G:[000664/008000], loss_rec=0.063
pre_G:[000665/008000], loss_rec=0.062
pre_G:[000666/008000], loss_rec=0.065
pre_G:[000667/008000], loss_rec=0.064
pre_G:[000668/008000], loss_rec=0.063
pre_G:[000669/008000], loss_rec=0.064
pre_G:[000670/008000], loss_rec=0.065
pre_G:[000671/008000], loss_rec=0.061
pre_G:[000672/008000], loss_rec=0.064
pre_G:[000673/008000], loss_rec=0.064
pre_G:[000674/008000], loss_rec=0.059
pre_G:[000675/008000], loss_rec=0.065
pre_G:[000676/008000], loss_rec=0.062
pre_G:[000677/008000], loss_rec=0.061
pre_G:[000678/008000], loss_rec=0.062
pre_G:[000679/008000], loss_rec=0.070
pre_G:[000680/008000], loss_rec=0.064
pre_G:[00068

pre_G:[000871/008000], loss_rec=0.057
pre_G:[000872/008000], loss_rec=0.058
pre_G:[000873/008000], loss_rec=0.055
pre_G:[000874/008000], loss_rec=0.058
pre_G:[000875/008000], loss_rec=0.055
pre_G:[000876/008000], loss_rec=0.061
pre_G:[000877/008000], loss_rec=0.058
pre_G:[000878/008000], loss_rec=0.060
pre_G:[000879/008000], loss_rec=0.061
pre_G:[000880/008000], loss_rec=0.059
pre_G:[000881/008000], loss_rec=0.061
pre_G:[000882/008000], loss_rec=0.056
pre_G:[000883/008000], loss_rec=0.059
pre_G:[000884/008000], loss_rec=0.064
pre_G:[000885/008000], loss_rec=0.061
pre_G:[000886/008000], loss_rec=0.059
pre_G:[000887/008000], loss_rec=0.064
pre_G:[000888/008000], loss_rec=0.058
pre_G:[000889/008000], loss_rec=0.063
pre_G:[000890/008000], loss_rec=0.059
pre_G:[000891/008000], loss_rec=0.060
pre_G:[000892/008000], loss_rec=0.057
pre_G:[000893/008000], loss_rec=0.060
pre_G:[000894/008000], loss_rec=0.061
pre_G:[000895/008000], loss_rec=0.060
pre_G:[000896/008000], loss_rec=0.059
pre_G:[00089

In [ ]:
#target
processing 225
9/231 too short (<128)
processing 226
1/356 too short (<128)
processing 227
1/389 too short (<128)
processing 228
8/366 too short (<128)
processing 229
38/379 too short (<128)
processing 230
5/397 too short (<128)
processing 231
55/456 too short (<128)
processing 232
54/412 too short (<128)
processing 233
8/372 too short (<128)
processing 236
39/492 too short (<128)
processing 239
11/503 too short (<128)
processing 240
25/377 too short (<128)
processing 243
5/393 too short (<128)
processing 244
0/420 too short (<128)
processing 250
37/481 too short (<128)
processing 254
52/397 too short (<128)
processing 256
0/319 too short (<128)
processing 257
105/434 too short (<128)
processing 258
56/414 too short (<128)
processing 259
4/481 too short (<128)
processing 267
14/417 too short (<128)
processing 268
0/406 too short (<128)
processing 269
19/396 too short (<128)
processing 270
22/462 too short (<128)
processing 273
17/433 too short (<128)
processing 274
3/466 too short (<128)
processing 276
6/462 too short (<128)
processing 277
49/459 too short (<128)
processing 278
6/409 too short (<128)
processing 279
18/405 too short (<128)
processing 282
7/368 too short (<128)
processing 286
8/468 too short (<128)
processing 287
2/424 too short (<128)

#8bit
processing 225
9/231 too short (<128)
processing 226
1/356 too short (<128)
processing 227
1/389 too short (<128)
processing 228
6/366 too short (<128)
processing 229
30/379 too short (<128)
processing 230
5/397 too short (<128)
processing 231
40/456 too short (<128)
processing 232
48/412 too short (<128)
processing 233
5/372 too short (<128)
processing 236
31/492 too short (<128)
processing 239
8/503 too short (<128)
processing 240
19/377 too short (<128)
processing 243
5/393 too short (<128)
processing 244
0/420 too short (<128)
processing 250
29/481 too short (<128)
processing 254
39/397 too short (<128)
processing 256
0/319 too short (<128)
processing 257
92/434 too short (<128)
processing 258
53/414 too short (<128)
processing 259
3/481 too short (<128)
processing 267
7/417 too short (<128)
processing 268
0/406 too short (<128)
processing 269
19/396 too short (<128)
processing 270
19/462 too short (<128)
processing 273
12/433 too short (<128)
processing 274
1/466 too short (<128)
processing 276
5/462 too short (<128)
processing 277
45/459 too short (<128)
processing 278
5/409 too short (<128)
processing 279
16/405 too short (<128)
processing 282
7/368 too short (<128)
processing 286
4/468 too short (<128)
processing 287
2/424 too short (<128)